In [1]:
import pandas as pd
import numpy as np
import time

import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt

import sys

!{sys.executable} -m pip install duckdb-engine

!{sys.executable} -m pip install ipython-sql

import duckdb, sqlalchemy

from sklearn.linear_model import LogisticRegression, LinearRegression

import statsmodels.api as sm

import statsmodels.formula.api as smf


%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [3]:
#storing the .csv files as dataframes
county_crime_dataset = pd.read_csv('Index_Crimes_by_County_and_Agency__\
Beginning_1990.csv')
nypd_arrests_dataset = pd.read_csv('NYPD_Arrests_Data__Historic_.csv')

We converted the dates in the ARREST_DATE column to DateTime values so we could easily access the time of the arrests. Also, because the reports in county_crime_dataset are listed by year, we added a YEAR column to nypd_arrests_dataset to more easily compare statistics among the two datasets. 

In [4]:
nypd_arrests_dataset['ARREST_DATE'] = pd.to_datetime(nypd_arrests_dataset['ARREST_DATE'], format='%m/%d/%Y')
nypd_arrests_dataset['YEAR'] = nypd_arrests_dataset['ARREST_DATE'].dt.year
nypd_arrests_dataset.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat,YEAR
0,236791704,2021-11-22,581.0,NaN,NaN,NaN,PL 2225001,M,M,28,0.0,45-64,M,BLACK,997427.0,230378.0,40.799009,-73.952409,POINT (-73.95240854099995 40.799008797000056),2021
1,237354740,2021-12-04,153.0,RAPE 3,104.0,RAPE,PL 1302502,F,B,41,0.0,25-44,M,WHITE HISPANIC,1013232.0,236725.0,40.816392,-73.895296,POINT (-73.89529641399997 40.816391847000034),2021
2,236081433,2021-11-09,681.0,"CHILD, ENDANGERING WELFARE",233.0,SEX CRIMES,PL 2601001,M,Q,113,0.0,25-44,M,BLACK,1046367.0,186986.0,40.679700,-73.776047,POINT (-73.77604736799998 40.67970040800003),2021
3,32311380,2007-06-18,511.0,"CONTROLLED SUBSTANCE, POSSESSION 7",235.0,DANGEROUS DRUGS,PL 2200300,M,Q,27,1.0,18-24,M,BLACK,NaN,NaN,NaN,NaN,NaN,2007
4,192799737,2019-01-26,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,0.0,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109,POINT (-73.94110928599997 40.800694331000045),2019


Since our datasets covered different time periods, we wanted to make our datasets start and end at the same time.

In [5]:
county_2008 = county_crime_dataset[(county_crime_dataset \
['Year'] >= 2008) & (county_crime_dataset \
['Year'] <= 2021)]
county_crime_dataset.head()
county_crime_dataset.tail()

,County,Agency,Year,Months Reported,Index Total,Violent Total,Murder,Rape,Robbery,Aggravated Assault,Property Total,Burglary,Larceny,Motor Vehicle Theft,Region
21818,Yates,Yates County State Police,2017,12.0,22.0,6.0,0.0,3.0,0.0,3.0,16.0,1.0,15.0,0.0,Non-New York City
21819,Yates,Yates County State Police,2018,12.0,35.0,3.0,0.0,3.0,0.0,0.0,32.0,4.0,27.0,1.0,Non-New York City
21820,Yates,Yates County State Police,2019,12.0,20.0,6.0,0.0,5.0,0.0,1.0,14.0,3.0,11.0,0.0,Non-New York City
21821,Yates,Yates County State Police,2020,12.0,26.0,3.0,0.0,2.0,0.0,1.0,23.0,3.0,20.0,0.0,Non-New York City
21822,Yates,Yates County State Police,2021,12.0,12.0,3.0,0.0,3.0,0.0,0.0,9.0,0.0,7.0,2.0,Non-New York City


In [6]:
nypd_arrests_2008 = nypd_arrests_dataset[(nypd_arrests_dataset \
['YEAR'] >= 2008) & (nypd_arrests_dataset \
['YEAR'] <= 2021)]
nypd_arrests_2008.head()
nypd_arrests_2008.tail()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat,YEAR
4801632,37566445,2008-01-08,567.0,"MARIJUANA, POSSESSION 4 & 5",235.0,DANGEROUS DRUGS,PL 2211001,M,Q,114,0.0,25-44,M,WHITE,1006898.0,220358.0,40.771487,-73.918235,POINT (-73.91823481899996 40.771487379000064),2008
4801633,37565060,2008-01-08,511.0,"CONTROLLED SUBSTANCE, POSSESSION 7",235.0,DANGEROUS DRUGS,PL 2200300,M,B,41,0.0,45-64,M,WHITE HISPANIC,1015927.0,236094.0,40.814651,-73.885563,POINT (-73.88556301199996 40.81465067700003),2008
4801638,37543923,2008-01-07,205.0,"TRESPASS 2, CRIMINAL",352.0,CRIMINAL TRESPASS,PL 1401500,M,B,42,0.0,<18,M,BLACK,1016142.0,243637.0,40.835353,-73.884750,POINT (-73.88475043899997 40.83535328100004),2008
4801644,37640214,2008-01-11,478.0,"THEFT OF SERVICES, UNCLASSIFIED",343.0,OTHER OFFENSES RELATED TO THEFT,PL 1651503,M,K,73,0.0,<18,M,BLACK,1005571.0,181640.0,40.665219,-73.923148,POINT (-73.92314834999998 40.66521910700004),2008
4801646,37688843,2008-01-14,905.0,"INTOXICATED DRIVING,ALCOHOL",347.0,INTOXICATED & IMPAIRED DRIVING,VTL11920M2,M,Q,109,0.0,18-24,M,ASIAN / PACIFIC ISLANDER,1030489.0,214264.0,40.754669,-73.833106,POINT (-73.83310572199997 40.75466902800008),2008


We dropped the columns that did not directly relate to our research questions. For example, many of the columns concerning internal/legal codes were dropped from the nypd_arrests dataset.

In [7]:
county_drop_cols = ['Months Reported']
county_crime = county_2008.drop(county_drop_cols, axis = 1)
nypd_arrests_drop_cols = ['ARREST_KEY','PD_CD', 'PD_DESC', 'KY_CD', 'LAW_CODE', 'LAW_CAT_CD', 'ARREST_PRECINCT', 'JURISDICTION_CODE']
nypd_arrests = nypd_arrests_2008.drop(nypd_arrests_drop_cols, axis = 1)

In [8]:
county_crime.to_csv('county_crime.csv')
nypd_arrests.to_csv('nypd_arrests.csv')